# This is a Test Report for Instacart Datasets based on Spotlight 

## Data Preprocessing (to Sequence data)

In [1]:
import os
import random

import numpy as np
import pandas as pd

DEFAULT_USER_COL = "user_ids"
DEFAULT_ITEM_COL = "item_ids"
DEFAULT_ORDER_COL = "order_ids"
DEFAULT_RATING_COL = "ratings"
DEFAULT_LABEL_COL = "label"
DEFAULT_TIMESTAMP_COL = "timestamp"
DEFAULT_PREDICTION_COL = "prediction"
DEFAULT_FLAG_COL = "flag"
data_base_dir = "./datasets/instacart_25/"
%matplotlib inline

In [2]:
products_file = data_base_dir + "raw/products.csv"
products = pd.read_csv(
    products_file, usecols=["product_id", "product_name", "aisle_id", "department_id"]
)
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [3]:
word_list = products["product_name"].unique()
len(word_list)

49688

In [4]:
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download("punkt")
nltk.download("stopwords")

stop_words = set(list(stopwords.words("english")) + list(string.punctuation))

new_stopwords = ["'s", "1", "2", "3", "4", "5", "6", "7", "8", "9", "And", "100"]
new_stopwords_list = stop_words.union(new_stopwords)

len(new_stopwords_list)

[nltk_data] Downloading package punkt to /home/zm324/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/zm324/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


223

In [5]:
vocab = []
for words in word_list:
    words = word_tokenize(words)
    for w in words:
        if w not in new_stopwords_list:
            vocab.append(w)

In [6]:
len(vocab)

223236

In [7]:
from nltk.probability import FreqDist

In [8]:
fdist1 = FreqDist(vocab)
len(fdist1)

12671

In [9]:
freq_vocab = []
for item in fdist1:
    if fdist1[item] >= 256:
        freq_vocab.append(item)
type(freq_vocab), len(freq_vocab)

(list, 164)

In [10]:
freq_vocab_dic = {}
index = 0
for item in freq_vocab:
    freq_vocab_dic[item] = index
    index += 1

In [11]:
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelEncoder

# label_encoder = LabelEncoder()
# integer_encoded = label_encoder.fit_transform(freq_vocab)
# print(integer_encoded)
# # binary encode
# onehot_encoder = OneHotEncoder(sparse=False)
# integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
# onehot_encoded = onehot_encoder.fit_transform(np.array(freq_vocab).reshape(len(freq_vocab), -1))
# print(onehot_encoded)

In [12]:
len(products["department_id"].unique())

21

In [13]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [14]:
aisles_file = data_base_dir + "raw/aisles.csv"
aisles = pd.read_csv(aisles_file, usecols=["aisle_id", "aisle"])
aisle_id_list = aisles["aisle_id"].unique()
n_aisle = len(aisle_id_list)
aisles_dict = {}
aisle_one_hot = np.eye(n_aisle)
for i in range(n_aisle):
    aisles_dict[aisle_id_list[i]] = aisle_one_hot[i]
len(aisles_dict)

134

In [15]:
department_file = data_base_dir + "raw/departments.csv"
department_df = pd.read_csv(department_file, usecols=["department_id", "department"])
department_id_list = department_df["department_id"].unique()
n_department = len(department_id_list)
departments_dict = {}
department_one_hot = np.eye(n_department)
for i in range(n_department):
    departments_dict[department_id_list[i]] = department_one_hot[i]
len(departments_dict)

21

In [16]:
product_ids = products["product_id"].unique()
filter_list = [" Chips"]
product_ids = np.array([x for x in product_ids if x not in filter_list])
product_ids.shape

(49688,)

In [22]:
features = {}
# count = 0
for index, _row in products.iterrows():
    _product_id = _row["product_id"]
    _product_name = _row["product_name"]
    _aisle_id = _row["aisle_id"]
    _department_id = _row["department_id"]
    if _product_id in product_ids:
        name_vec = aisles_dict[_aisle_id]
        name_vec = np.concatenate([name_vec, departments_dict[_department_id]])
        features[_product_id] = name_vec
#     print(features[_product_id])
len(features)

49688

In [23]:
fea_out_file = data_base_dir + "raw/item_feature_cate.csv"
fea_out = open(fea_out_file, "w")
fea_out.write("item_id,feature\n")
for _product_id in product_ids:
    fea_out.write(
        str(_product_id) + "," + " ".join(map(str, features[_product_id])) + "\n"
    )
#     break
fea_out.close()

In [24]:
len(features[3])

155

In [ ]:
len(freq_vocab_dic)

In [21]:
np.concatenate([name_vec, name_vec])

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.